In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from functools import partial
import sklearn
from tqdm import tqdm
import sklearn.metrics 
from IPython.display import clear_output

In [3]:
import sys
from matplotlib import pyplot as plt
import seaborn as sns
import sys
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import FloatType, IntegerType, LongType
from recommenders.utils.spark_utils import start_or_get_spark
from IPython.display import display


print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("PySpark version: {}".format(pyspark.__version__))

System version: 3.7.13 (default, Mar 28 2022, 08:03:21) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.3.5
PySpark version: 3.1.2


## 1 Recommender Systems as a  Bandit problem

Recommender systems are sequential decision problems since users interact with items over a period of time and reveal their preferances.The
fundamental explore/exploit tradeoff (a feature of  multi-armed bandits/Reinforcement Learning) exists in recommender systems as the recommender
agent must choose between recommending the highest rated item to each user as per its current state of knowledge (exploit) and recommending items
which potentially increases the agent's knowledge (reduce uncertainty)  about the preferences of users and attributes of items (explore).The training data available to the agent to update its knowledge of users/items preferences is dependent upon the recommendations it served in the past.This is a feature of Reinforcement Learning (RL) rather than pure  Supervised Learning.A simple recommender agent interface would be as follows:

In [5]:
class RecommenderInterface:
    """Interface for recommender agents"""
    def get_recommendations(self, userids:np.ndarray) -> pd.DataFrame:
        """ Perform Inference,I.E give top recommended item for each user in userids,based of current
           user/item knowledge"""
        pass

    def learn(self, interactions:pd.DataFrame,*args,**kwargs) -> None:
        """Update User/Item attributes based on xplicit user feedback of recommendations made"""
        pass

## 2 Collaborative Filtering

The Recommender Agent can learn user preferances and item attributes through multiple ways. The two popular approaches are collaborative filtering and content based filtering. In collaborative filtering (which we will be focusing on),the agent learns user preferances and item
attributes through explicit user feedback (ratings) of the recommended items.A common approach is to use Matrix Factorization algorithms to 
find latent factors in a low dimension that represent user/item attributes,the idea is to uncover such factors which match the observed ratings
given by the users to various recommended items.


### 2.1 Matrix factorization for collaborative filtering problem

Matrix factorization is a common technique used in recommendation tasks. Basically, a matrix factorization algorithm tries to find latent factors that represent intrinsic user and item attributes in a lower dimension. That is,

$$\hat r_{u,i} = q_{i}^{T}p_{u}$$

where $\hat r_{u,i}$ is the predicted ratings for user $u$ and item $i$, and $q_{i}^{T}$ and $p_{u}$ are latent factors for item and user, respectively. The challenge to the matrix factorization problem is to find $q_{i}^{T}$ and $p_{u}$. This is achieved by methods such as matrix decomposition. A learning approach is therefore developed to converge the decomposition results close to the observed ratings as much as possible. Furthermore, to avoid overfitting issue, the learning process is regularized. For example, a basic form of such matrix factorization algorithm is represented as below.

$$\min\sum(r_{u,i} - q_{i}^{T}p_{u})^2 + \lambda(||q_{i}||^2 + ||p_{u}||^2)$$

where $\lambda$ is a the regularization parameter. 


### 2.2 Alternating Least Square (ALS)

Owing to the term of $q_{i}^{T}p_{u}$ the loss function is non-convex. Gradient descent method can be applied but this will incur expensive computations. An Alternating Least Square (ALS) algorithm was therefore developed to overcome this issue. 

The basic idea of ALS is to learn one of $q$ and $p$ at a time for optimization while keeping the other as constant. This makes the objective at each iteration convex and solvable. The alternating between $q$ and $p$ stops when there is convergence to the optimal. It is worth noting that this iterative computation can be parallelised and/or distributed, which makes the algorithm desirable for use cases where the dataset is large and thus the user-item rating matrix is super sparse (as is typical in recommendation scenarios). A comprehensive discussion of ALS and its distributed computation can be found [here](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf).


## 2.3 ALS based Recommender agent

Following is an implementation of a recommender agent which uses the ALS Algorithm to learn user/item latent factors ($q_{i}^{T}$ and $p_{u}$) from ratings data.Note that in order to serve recommendations at any point in time,the agent recommends the item with the highest predicted rating for each user.This implies that the agent is implementing a 'greedy' policy i.e exploiting its state of knowledge at each step,without making exploratory recommendations.This results in sub-optimal performance (as we will see later).The greedy policy can be improved by using an
$\epsilon$-greedy approach,i.e with a small probability $\epsilon$ the agent will recommend a random item from the catalogue to a user and with a probability 1-$\epsilon$ it will recommend the top rated item.This ensures continuous exploration but has the drawback that the agent continues to make sub-optimal recos even after learning a lot about user/item attributes.

In [6]:
class ALSRecommender(RecommenderInterface):
    """Wrapper class to use ALS algorithm to fit user/item latent factors to interaction data
       and serve top recommendations for requested userids (run inference) using model state
       """

    def __init__(self,n_items,**kwargs):
        """ Args:
            als = ALS(
            maxIter=1, : ALS iterations- kept as 1 to match bayesian recommender updates
            rank=10,  :  number of latent factors (n_features)
            regParam=0.05, : regularization hyperparam
         )
         """
        
        ## store itemids
        self.itemids=np.array(range(n_items))
        
        ## init spark session
        findspark.init()
        self.spark = start_or_get_spark("ALS Deep Dive", memory="8g")
        self.spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")
        
        ## init ALS model class with hyperparams passed down from caller
        self.als = ALS(**kwargs,
                    userCol='userid', 
                    itemCol='itemid', 
                    ratingCol='actual_rating', 
                    coldStartStrategy='drop')
        
        ## init  spark schema to convert pd dfs to spark dfs
        self.inference_schema = StructType((
        StructField('userid', IntegerType()),
        StructField('itemid', IntegerType()),
           ))
        self.learning_schema=StructType((
        StructField('userid', IntegerType()),
        StructField('itemid', IntegerType()),
        StructField('actual_rating', FloatType()),
        
           ))
        
        ## since coldStartStrategy="drop" for the ALS model,initially (when no user/item)
        ## interactions are observed,model cannot predict ratings and hence random recomme-
        ##-ndations are served.the flag is set to false as soon as ALSRecALSRecommender.learn()
        ## is called
        self.random_initial_recos=True
       
        ## since the ALS model does not support incremental training (as new interaction data
        ## comes in) .. we append the data into an ever increasing dataset and fit on all the  
        ## interactions available at any given time.
        self.interaction_data=pd.DataFrame()
        
    def get_recommendations(self,userids:np.ndarray)->pd.DataFrame:
        
        if self.random_initial_recos:
            rec_items=np.random.choice(self.itemids,size=len(userids),replace=True)
            pred_ratings=np.NAN
            return pd.DataFrame({'userid':userids,'itemid':rec_items,'predicted_rating':pred_ratings})
        
        else:
            ## run inference on all user/item pairs and recommend the highest rated item for each user
            user_items=pd.DataFrame({'userid':userids}).merge(pd.DataFrame({'itemid':self.itemids}),
                                     how='cross')
            ## convert to spark df to run inference with ALS model
            infer_df=self.spark.createDataFrame(user_items,schema=self.inference_schema) 
            ## inference step
            pred_df=self.model.transform(infer_df)
            ## rename inferred (prediction) column -coming from ALS implementation to predicted_rating
            ## to make it consistent
            pred_df=pred_df.withColumnRenamed("prediction","predicted_rating")
            ## convert predictions back to a pandas df
            pred_df=pred_df.toPandas()
            ## recommend top item for each user,ranked by predicted rating
            recos=pred_df.loc[pred_df.groupby('userid')['predicted_rating'].idxmax()]
            
        
            return recos

            
            
    def learn(self,interactions:pd.DataFrame)->None:
        ## set random recommendations flag to false when learning begins
        self.random_initial_recos=False
        self.interaction_data=pd.concat([self.interaction_data,interactions])
        ## convert to spark df to fit als model
        interactions=self.spark.createDataFrame(self.interaction_data[['userid','itemid','actual_rating']]
                                                ,schema=self.learning_schema) 
        
        ## fit model on all interaction data available accumalated this time
        self.model=self.als.fit(interactions)
        
        
        
        
        
        
        
        
        
       
            
            
        
    
    
        
    

# 3 Bayesian Recommender agent

In order to address the explore/exploit tradeoff in the recommender system setting,we propose a bayesian update strategy to update the agent's
beliefs about user/item attributes from ratings data.The key difference here is that instead of learning user/item latent vectors ($q_{i}^{T}$ and $p_{u}$),we learn (multivariate-gaussian) distributions for each user and item, i.e  $q_{i}$ ~ N($\mu$_i,$\sigma$_i) and  $p_{u}$ ~ N($\mu$_u,$\sigma$_u).This allows us to sample user/item distributions to predict ratings and recommend the highest rated item.

The main advantage of this approach is that the agent makes recommendations that either:

* It is confident would result in a high rating
* It has high uncertainity about (high variance of user/item attributes),this helps in exploring possibly better recommendations.

This approach is inspired from Thompson Sampling for bernoulli bandits,which uses bayesian updates on a prior beta distribution to sample actions. More details of which can be found [here](https://web.stanford.edu/~bvr/pubs/TS_Tutorial.pdf).


## 3.1 Bayesian Linear Regression

The primary building block of our bayesian recommender system is the update mechanism which performs updates to user/item latent variable distributions from observed interactions i.e 

if we have prior beliefs about user u and item i,such that their latent attributes are distributed as:$$\hat p_{u} ~ N(\mu_u,\Sigma_u)$$ and
$$\hat q_{i} ~ N(\mu_i,\Sigma_i)$$ and we observe an interaction  which results in a rating $r_{u,i}$  where $$\hat r_{u,i} = q_{i}^{T}p_{u} + \epsilon$$ 

where $\epsilon$ is a noise term with known std. deviation = $\beta$,then the posterior distributions of $p_{u}$ | $q_{i}$,$r_{u,i}$
and  $q_{i}$ | $p_{u}$,$r_{u,i}$ will be gaussian too. The new means and variances $\mu_u^*$,$\Sigma_u^*$ and $\mu_i^*$,$\Sigma_i^*$ are related 
to the original - $\mu_u$,$\sigma_u$ and $\mu_i$,$\sigma_i$ as follows:

## 3.2 Bayesian Update

Suppose we have to update our  **user** attribute distribution given an interaction $I_{u,i}$ between user $u$ and item $i$ resulting in a rating 
$r_{u,i}$,with prior attribute distributions $ N(\mu_i,\Sigma_i)$ and $ N(\mu_u,\Sigma_u)$ respectively, first we draw a dataset $\Phi$ comprising of n_samples of possible **item** attributes sampled from $ N(\mu_i,\Sigma_i)$ ,then the posterior distribution of our **user** attributes $p_{u}$|$r_{u,i}$, $\Phi$,$\mu_u$,$\Sigma_u$,$\beta$ ~ N($\mu_u^*$,$\Sigma_u^*$) is defined by : 

$$Inv(\Sigma_u^*)=Inv(\Sigma_u)+\beta\Phi^{T}\Phi$$  and $$\mu_u^*=\Sigma_u^*(Inv(\Sigma_u)\mu_u+\beta\Phi^{T}r_{u,i})$$ 

Where $Inv()$ is the matrix inverse operator. These equations corospond to the update rules for bayesian linear regression as decribed in the 
textbook Pattern Recognition and Machine Learning by Christopher M. Bishop (eqs 3.50,3.51) as found here-http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf. 

After updating our beliefs about our **user** attribute distribution,we follow the same process interchanging the **user** and **item** distributions to make a similar update to our **item** distribution . Note that we use the updated **user** distribution from the last step to
make our **item** update. This (**user** and **item**) update  completes one *iteration*

A simple implementation of a distribution class that can be sampled from and can make the aforementioned Bayesian updates is as follows:

In [7]:
class BayesianDistribution():

    def __init__(self, n_features=10, alpha=10., beta=0.1,**kwargs):
        self.n_features = n_features
        self.alpha = alpha
        self.beta = beta
        self.mean = np.zeros(n_features)
        self.cov_inv = np.identity(n_features) / alpha
        self.cov=np.identity(n_features) * alpha
    
    def sample(self,n_samples:int)->np.ndarray:
        return np.random.multivariate_normal(mean=self.mean,cov=self.cov,size=n_samples)
        
    

    def learn(self, X, y):
        # If x and y are singletons, then we coerce them to a batch of length 1
        X = np.atleast_2d(X)
        y = np.atleast_1d(y)
        
      
        # Update the inverse covariance matrix (Bishop eq. 3.51)
        cov_inv_post = self.cov_inv + self.beta * X.T@X

        # update covariance by inverting posterior cov_inv
        self.cov = np.linalg.inv(cov_inv_post)
         # Update the mean vector (Bishop eq. 3.50)
      
        mean = self.cov @ (self.cov_inv @ self.mean + self.beta *X.T@y)
        self.cov_inv = cov_inv_post
        self.mean = mean

        
    def predict_mean(self,X):
        return X@self.mean

    def predict(self, x):

        # Obtain the predictive mean (Bishop eq. 3.58)
        y_pred_mean = x @ self.mean

        # Obtain the predictive variance (Bishop eq. 3.59)
        w_cov = np.linalg.inv(self.cov_inv)
        y_pred_var = 1 / self.beta + x @ w_cov @ x.T

        return stats.norm(loc=y_pred_mean, scale=y_pred_var ** .5)
    
    ## just so alling code can refer to wts dist as BayesianDistribution().weights_dist
    @property
    def weights_dist(self):
        cov = np.linalg.inv(self.cov_inv)
        return stats.multivariate_normal(mean=self.mean, cov=cov)
    
 
    def run_bayesian_reg_test(self,x_count,max_iters):
        np.random.seed(1234)
        dist_learner=BayesianDistribution(n_features=self.n_features,alpha=self.alpha,beta=self.beta)
        true_wts=np.random.rand(*dist_learner.mean.shape)
        res=[]
        for i in range(max_iters):
            # create random data
            X=np.random.rand(x_count,dist_learner.n_features)
            y=X@true_wts+np.random.randn(x_count,1)*dist_learner.beta
            
            # calc metrics to track
            y_pred=dist_learner.predict_mean(X)
            target_mse=sklearn.metrics.mean_squared_error(y,y_pred)
            wts_mse=sklearn.metrics.mean_squared_error(true_wts,dist_learner.mean)
            
            row={'iteration':i,'target_MSE':target_mse,'features_MSE': wts_mse,
                 'true_features':true_wts,'estimated_features':dist_learner.mean}
            res.append(row)
            
            ## learning is inplace,modifying the prior_dist variable
            dist_learner.learn(X,y)
         
        return pd.DataFrame(res)  
            
           
           
            
       
    

## 3.3 Bayesian Recommender Algorithm:
* Initialize user,item attribute distributions with uninformative priors ($\mu$=0, $\Sigma$=large)
    <br>
* for t in n_timeperiods do:
> * Receive recommendation requests for a list of users 
> * sample n=1 samples for user,item attributes ( $p_{u}$ and $q_{i}$) for all users in requests and items in the catelogue using
> the current user/item distributions $ N(\mu_i,\Sigma_i)$ and $ N(\mu_u,\Sigma_u)$
> * Recommend to each user u, an item i which has the highest predicted rating i.e i=argmax($q_{i}^{T}p_{u}$)
> * Receive ratings $r_{u,i}$ for each recommended pair user u,item i ,these form the training data or ground truth with which to update
> user/item distributions.
> * for each $r_{u,i}$ in ratings do:
>> * for i in n_iterations:
>>> * Sample **item** attributes from current item attribute distribution ($ N(\mu_i,\Sigma_i)$),store samples in dataset $\Phi$
>>> * Update **user** attribute distribution using the Bayesian Update rule decribed in sec 3.2 
<br>
>>> i.e $\mu_u^*$,$\Sigma_u^*$->BayesianUpdate($r_{u,i}$,$\Phi$,$\beta$,$\mu_u$,$\Sigma_u$)
>>> * Sample **user** attributes from current user attribute distribution ($ N(\mu_u^*,\Sigma_u^*)$),store samples in dataset $\Phi^*$
>>> * Update **item** attribute distribution using the Bayesian Update rule decribed in sec 3.2 
<br>
>>> i.e $\mu_i^*$,$\Sigma_i^*$->BayesianUpdate($r_{u,i}$,$\Phi^*$,$\beta$,$\mu_i$,$\Sigma_i$)






A Recommender agent implementing the above algorithm would be as follows:

In [8]:
class BayesianRecommender(RecommenderInterface):

    def __init__(self,n_items, dist_cls=BayesianDistribution,epsilon_greedy=False,**kwargs):
        """Bayesian recommender class.Internally store and update distributions over user and item
           features(embeddings) Args:
           
           dist_cls:parametrized distribution class representing current belief of user/item attributes
                    which is updatable with incoming ratings data
           
           dist_kwargs:kwargs to init dist_cls. represent prior belief for user/items for which no
                       ratings data is yet available
            """
        
        init_user_item_func=partial(dist_cls,**kwargs)
       
        ## defaultdicts to initialise users/items which have no interaction data with uninformative priors
        ## i.e large covariance matrices, key: user/item id ,value: instance of the distribution class
        ##used to represent belief over user/item attributes ,given the ratings data so far. 
       
        self.USERS=defaultdict(init_user_item_func)
        self.ITEMS=defaultdict(init_user_item_func)
        self.itemids=np.array(range(n_items))
        
        
    def get_recommendations(self,userids:np.ndarray)->pd.DataFrame:
        
        ## sampling user and item attributes from respective distributions,
        ## and then recommending item with highest predicted rating. This is akin
        ## to thompson sampling
        user_attrs=np.stack([self.USERS[userid].sample(1).flatten() for userid in userids],axis=1)
        item_attrs=np.stack([self.ITEMS[itemid].sample(1).flatten() for itemid in self.itemids],axis=1)
        
        ## calculating expected ratings for requested userids and all items in inventory 
        ratings_mat=item_attrs.T@user_attrs
        
        ## recommend top item as per predicted ratings
        rec_items=self.itemids[np.argmax(ratings_mat,axis=0)]    
        pred_rating=np.max(ratings_mat,axis=0)
        
        
        recos=pd.DataFrame({'userid':userids,'itemid':rec_items,'predicted_rating':pred_rating})
        return recos


    def learn(self,interactions:pd.DataFrame,n_iters=1,n_samples=1000)->None:
        
        ## do bayesian updates to each user/item distributions for each user/item interaction based on actual interaction data
        for userid,itemid,target_score in zip(interactions['userid'],interactions['itemid'],interactions['actual_rating']):
            
            user_dist,item_dist=self.USERS[userid],self.ITEMS[itemid]
            target_score=np.repeat(target_score,n_samples)
            
            for i in range(n_iters):
                ## first update user distribution while sampling from the item dist
                user_dist.learn(item_dist.sample(n_samples),target_score)
               
                ## then update item distribution while sampling from the updated user dist
                item_dist.learn(user_dist.sample(n_samples),target_score)
                ## both updates increase the mean likelyhood of the interaction data given
                ## user/item beliefs.The likelyhood is upper bounded by the inner product
                ## of the true user-item attributes,hence with enough interactions the
                ## mean of the user/item distributions should converge to the true attribute
                ## vector for all users and items
               
            
            
        
    
    
        
    

# 4 Experiment Design

In order to evaluate our recommender agents in the sequential setting (when users preferences and item attributes are revealed over time),we use
a user/item simulator instead of a static dataset of user/item IDs and corresponding ratings.This allows us to not only compute standard evaluation metrics (like M.S.E error) between predicted and actual ratings, but also the **Regret** which is the difference in ratings between a the highest rated item and the recommended item for any user. A decrease in **Regret** would imply that the recommender was making better recommendations

## 4.1 User/Item Simulator

The User/Item simulator contains the **TRUE** user and item attributes ($Q_{i}$ and $P_{i}$),then the **TRUE** assigned rating $r_{u,i}$|$Q_{i}$,$P_{i}$
is given by
$$\ r_{u,i} = Q_{i}^{T}P_{u} + \epsilon$$ 
where $\epsilon$ is the ratings noise. The **TRUE** attributes $Q_{i}$ and $P_{i}$ are initialized randomly to values between 0 and 1 and are hidden from the recommender agent.The agent must infer these attributes by using the ratings generated by the simulator for its recommendations.
Below is an implementation for such a user/Item Simulator

In [10]:
class UserItemSimulator():

    def __init__(self, n_users=100000,n_items=1000,n_features=10,rat_noise=0.1):
        """Class to simulate actual user/item interactions and generate recommendation requests Args:
           
           n_users:max number of users interacting with the system
           
           user_pct: %tage of users which request recommendation in one time period
           
           n_items: number of items in recommendation inventory
           
           rat_noise:s.d. of the noise in the expected rating,given user/item attrs
           
           i.e true_rating=user_attrs@item_attrs.T+noise
            """
    
        ## storing true user/item attrs in np.arrays of shape (n_users/items,n_features)
        ## this way they can be integer indexed as user_attrs=self.USER_ATTRS[userid] etc.
        
        self.userids=np.array(range(n_users))
        self.USER_ATTRS=np.random.rand(n_users,n_features)
        self.ITEM_ATTRS=np.random.rand(n_items,n_features)
        self.rating_noise=rat_noise
        
        ## precalculating true mean true ratings for all users and items
        self.TRUE_RATINGS_MAT=self.ITEM_ATTRS@self.USER_ATTRS.T
        
        ## precalculating best possible ratings for  optimal recommendations
        self.BEST_RECOS=np.max(self.TRUE_RATINGS_MAT,axis=0)
        
        
        
    def generate_requests(self,n_requests=10000):
        return np.random.choice(self.userids,size=n_requests,replace=False)
    
    def score_reccomendations(self,recos):
        recos['actual_rating_mean']=self.TRUE_RATINGS_MAT[recos['itemid'],recos['userid']]
        ## the actual ratings have a noise component to them,added to the mean...these are the ratings we use to learn
        recos['actual_rating']=  recos['actual_rating_mean']  +self.rating_noise*np.random.randn()
        recos['best_rating_mean']=self.BEST_RECOS[recos['userid']]
        ## use the mean actual ratings to compute regret so regret goes to 0 when we select the items with the highest average rating for each user 
        recos['regret']=recos['best_rating_mean']-recos['actual_rating_mean']
        return recos
     

## 4.2 Usage/Demo

### Init user/item simulator and Recommender agent 

In [11]:
## initing user-item simulator
user_sim=UserItemSimulator(n_users=1000,n_items=1000,n_features=10,rat_noise=0.1)
## init bayesian recommender
b_rec=BayesianRecommender(n_items=1000,n_features=10, alpha=10., beta=0.1)

### Get Recommendation Requests

In [12]:
requests=user_sim.generate_requests(1000)
pd.DataFrame({'userid':requests})

,userid
0,597
1,176
2,606
3,705
4,771
...,...
995,92
996,179
997,445
998,198


### Get recommendations for the user requests

In [13]:
recos=b_rec.get_recommendations(requests)
recos

,userid,itemid,predicted_rating
0,597,469,130.524879
1,176,541,38.697143
2,606,805,71.775643
3,705,810,92.562352
4,771,588,88.547633
...,...,...,...
995,92,547,49.742615
996,179,621,135.450418
997,445,124,105.282972
998,198,84,108.077796


### Score generated recommendations

In [18]:
scored_interactions=user_sim.score_reccomendations(recos)
scored_interactions

,userid,itemid,predicted_rating,actual_rating_mean,actual_rating,best_rating_mean,regret
0,237,291,102.563792,3.649996,3.642422,4.476816,0.826820
1,914,381,96.551606,2.485622,2.478049,4.539948,2.054326
2,484,446,119.845155,1.521432,1.513858,3.541389,2.019958
3,657,799,72.066049,1.523088,1.515515,3.274659,1.751570
4,144,839,59.076952,2.222614,2.215041,3.600775,1.378160
...,...,...,...,...,...,...,...
995,414,258,81.044592,2.944138,2.936564,4.770941,1.826804
996,131,306,129.513825,1.271351,1.263777,3.813208,2.541857
997,998,787,85.131933,2.201177,2.193603,3.964226,1.763049
998,604,242,102.116459,3.319046,3.311472,4.961711,1.642665


### Improve Recommender agent with the scored interactions

In [19]:
b_rec.learn(scored_interactions)

## Experiment Flowchart
![Experiment Flowchart](Recommender_expt.png)

## Helper Class to time code run

In [9]:
from timeit import default_timer


class Timer():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.timer = default_timer
        
    def __enter__(self):
        self.start = self.timer()
        return self
        
    def __exit__(self, *args):
        end = self.timer()
        self.elapsed = end - self.start

## 4.3 Final Experiment 
Below is the code for running the experiment for a specified number of periods and specified number of users and items (n_periods=30,50 etc.)

In [10]:
def run_expt(n_periods,n_users=10000,n_items=100):
    
    np.random.seed(42)
    
    ## initing user-item simulator
    user_sim=UserItemSimulator(n_users,n_items,n_features=10,rat_noise=0.1)
    ## init bayesian recommender
    b_rec=BayesianRecommender(n_items=n_items,n_features=10, alpha=10., beta=0.1)
    ## init ALS recommender
    a_rec=ALSRecommender(maxIter=1,rank=10,regParam=0.05,n_items=n_items)
    ## run recommend-score-update loop for n_periods
    log=[]
    for t in tqdm(range(n_periods)):
        ## generate recommendation requests
        requests=user_sim.generate_requests(n_users)
        ## get recommendations
        with Timer() as infer_timer_a:
            recos_a=a_rec.get_recommendations(requests)
            
        with Timer() as infer_timer_b:
            recos_b=b_rec.get_recommendations(requests)
            
        ## score recommendations
        interactions_a=user_sim.score_reccomendations(recos_a)
        interactions_b=user_sim.score_reccomendations(recos_b)
        
        ## update/fit recommenders with iteraction data
        with Timer() as train_timer_a:
            a_rec.learn(interactions_a)
            
        with Timer() as train_timer_b:
            b_rec.learn(interactions_b)
            
        ## insert scoring/mse error/regret tracking code here
        row={'mean_regret_ALS':interactions_a['regret'].mean(),
             'mean_regret_Bayesian':interactions_b['regret'].mean(),
             'ratings_mse_ALS':((interactions_a['predicted_rating']-interactions_a['actual_rating_mean'])**2).mean(),
             'ratings_mse_Bayesian':((interactions_b['predicted_rating']-interactions_b['actual_rating_mean'])**2).mean(),
             'inferance_time_ALS':infer_timer_a.elapsed,
            'inferance_time_Bayesian':infer_timer_b.elapsed,
             'training_time_ALS':train_timer_a.elapsed,
             'training_time_Bayesian':train_timer_b.elapsed
            }
        log.append(row)
        clear_output(wait=True)
        display(pd.DataFrame(log))
        
    
    return pd.DataFrame(log),user_sim,a_rec,b_rec
        
    
    
                              
        
    
                              
        
        
    

## Run and Results

In [ ]:
log,sim,a_rec,b_rec=run_expt(30,n_users=1000,n_items=1000)

,mean_regret_ALS,mean_regret_Bayesian,ratings_mse_ALS,ratings_mse_Bayesian,inferance_time_ALS,inferance_time_Bayesian,training_time_ALS,training_time_Bayesian
0,1.583331,1.568143,NaN,10104.148497,0.021003,1.113949,21.986770,3.406009
1,1.215116,1.201606,0.972556,2366.448140,40.985261,0.718517,7.316114,3.319364
2,1.180599,1.173342,0.215609,202.964224,40.007753,0.782418,6.447025,3.356611
3,1.101050,1.153942,0.087176,94.926841,37.693448,0.909922,6.518701,3.310713
4,1.082075,1.133270,0.063502,40.979157,38.159967,0.754672,6.906252,3.140443


 17%|█████████████▊                                                                     | 5/30 [03:47<19:52, 47.70s/it]

# 5 Results and Evaluation Metrics
## 5.1 Regret vs MSE

It is often noted that the mean square error (MSE) between predicted and actual user/item ratings ($\hat r_{u,i}$ and  $\ r_{u,i}$) is not 
a good metric to evaluate recommender systems. This is because the objective isn't to predict ratings with high accuracy,rather the objective
of a recommender system is to ensure that over time users are recommended items which are likely to be assigned high ratings,eventually recommending the highest rated item in the catelogue for each user. To track this objective, we use another metric taken from RL/Bandit literature called **Regret**. The **Regret** of any action (here,recommendation) is the difference in ratings of the best possible item 
and the recommended item for any user. A **Regret**  of 0 would imply the recommender is making the best possible recommendation for a given
user.


## 5.2 Explore/Exploit Tradeoff

In the results of our experiment run we can make the following observations:

* The **Regret**  for the ALS agent drops faster initially than the Bayesaian agent,however
  it eventually plateaus off and the Bayesian agent is able to reduce the mean **Regret** further.

* The **MSE**  for The ALS agent
    starts with a smaller initial value and quickly reduces it to near 0, whereas the Bayesian agent starts with a very high **MSE**,eventually    reducing it as it learns about the rating values.

The explanation for the observed behaviour is as follows:

* The ALS agent quickly overfits to the initial batches of ratings data,reducing **MSE** and becomes highly confident of its ratings
predictions too early as a result it eventually ends up only exploiting its state of knowledge and does not explore potentially better 
recommendations resulting in a plateued performance.

* The Bayesian agent starts off with a very high **MSE** (predicting ratings outside the possible range of values),this is because the user/item
attributes are being sampled from a high variance distribution.The variance of the user/item attribute distributions are only reduced after the
agent has tried many combinations users and items.This principle is called **optimism in the face of uncertainity** in Bandit literature and it ensures the recommender agent tries out various items for each user before converging on the optimal recommendation.At the same time the mean
of the user/item distributions are adjusted so the selection of higher rated items becomes increasingly likely.Thus the Bayesian agent is able 
to address the Explore/Exploit tradeoff in a more effective manner.


In [52]:
log,sim,a_rec,b_rec=run_expt(50,n_users=1000,n_items=1000)

,mean_regret_ALS,mean_regret_Bayesian,ratings_mse_ALS,ratings_mse_Bayesian,inferance_time_ALS,inferance_time_Bayesian,training_time_ALS,training_time_Bayesian
0,1.108242,0.995239,NaN,5605.170497,0.000967,0.131320,4.558881,1.282589
1,0.916315,0.950820,0.125324,12126.136869,5.300142,0.114389,4.785893,1.175667
2,0.899511,0.937947,0.029722,5870.117462,5.349249,0.108105,4.684738,1.153631
3,0.893283,0.982529,0.010572,4204.639763,5.250036,0.100655,4.691514,1.176227
4,0.887859,0.960208,0.003225,3207.297049,5.415202,0.105176,4.662004,1.154958
5,0.888595,0.931749,0.004195,2197.325646,5.258526,0.108297,4.737060,1.134376
6,0.887307,0.929820,0.005367,1579.098645,5.407707,0.101874,4.679289,1.174039
7,0.886859,0.911312,0.003748,1168.826586,5.312556,0.111625,4.682302,1.204642
8,0.887453,0.928927,0.004178,653.269551,5.339012,0.098653,4.704821,1.199471
9,0.885962,0.923586,0.002059,343.105181,5.440180,0.104991,4.701531,1.142707


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [09:37<00:00, 11.56s/it]
